In [1]:
#sql 연결 및 데이터 불러와서 df로 변환
import pandas as pd
import pymysql

with open('pentagon1004.sql', 'r') as f:
    sql_script = f.read()

with open(f"./db_config", "r") as f:
    config = f.readlines()

config_dict = {}
for l in config:
    idx = l.index('=')
    k = l[:idx]
    v = l[idx + 1:]
    config_dict[k] = v.rstrip()

conn = pymysql.connect(**config_dict)
cur = conn.cursor()
# cur.execute(sql_script)  # Assuming that your SQL script creates the table
df = pd.read_sql_query("SELECT * FROM pentagon_news", conn)


C:\Users\lonec\AppData\Local\Temp\ipykernel_56652\1809263890.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM pentagon_news", conn)


In [ ]:
# 데이터 확인
len(df)

In [2]:
# 샾 분리, 증권 제외
df['stock'] = df['mecab_noun_stock'].str.split('#')
df = df[['title', 'press', 'date_upload', 'mecab_noun_stock', 'url', 'stock']]
df['stock'] = df['stock'].apply(lambda x: [word for word in x if not word.endswith('증권')] if isinstance(x, list) else [])
df.head(10)

,title,press,date_upload,mecab_noun_stock,url,stock
0,"이복현#""소상공인#울리는#보이스피싱#엄단""",이데일리,2023-04-05,대웅,https://n.news.naver.com/mnews/article/018/000...,[대웅]
1,이석태·강신국·박완식·조병규…#우리은행장#4파전,디지털타임스,2023-03-26,대교#우리금융지주,https://n.news.naver.com/mnews/article/029/000...,"[대교, 우리금융지주]"
2,"KB금융,#중국#소비회복#가능성#엿본#보고서#발간",스포츠서울,2023-03-26,KB금융,https://n.news.naver.com/mnews/article/468/000...,[KB금융]
3,코람코에#1위#빼앗긴#한토신·한자신…올해도#'먹구름'#전망,더팩트,2023-03-31,한국토지신탁#카카오#한국자산신탁#한국기업평가,https://n.news.naver.com/mnews/article/629/000...,"[한국토지신탁, 카카오, 한국자산신탁, 한국기업평가]"
4,임승태#KDB생명#신임#대표이사#취임,서울경제,2023-03-31,KD#DB#CS,https://n.news.naver.com/mnews/article/011/000...,"[KD, DB, CS]"
5,BNK금융그룹#계열사#대표#확정…#6곳#중#5곳#‘새#인물’,부산일보,2023-03-26,대교#CS#BNK금융지주,https://n.news.naver.com/mnews/article/082/000...,"[대교, CS, BNK금융지주]"
6,"신보-기업은행,#복합위기#극복#지원#위한#상설#협의체#출범",데일리안,2023-03-31,기업은행,https://n.news.naver.com/mnews/article/119/000...,[기업은행]
7,대출한도#200만원#넘게#오른다고?…'신용점수#올리기'#앱#유행,뉴시스,2023-03-26,카카오#카카오페이,https://n.news.naver.com/mnews/article/003/001...,"[카카오, 카카오페이]"
8,"농협은행,#일회용품#제로#챌린지#동참",데일리안,2023-03-31,한화#E1#레이#SG,https://n.news.naver.com/mnews/article/119/000...,"[한화, E1, 레이, SG]"
9,"우수수#쏟아지는#청년펀드…""가입하면#뭐가#좋나요?""#[신민경의#편드는#펀드]",한국경제,2023-03-26,한화#디오,https://n.news.naver.com/mnews/article/015/000...,"[한화, 디오]"


In [3]:
# 같은 뉴스, 다른 언론사 제거
from tqdm import tqdm

with tqdm(total=len(df)) as pbar:
    df['stock_str'] = df['stock'].apply(lambda x: ','.join(x))
    df.drop_duplicates(subset=['date_upload', 'stock_str'], inplace=True)
    pbar.update(len(df))

df.drop(['stock_str'], axis=1, inplace=True)  # 임시 열 삭제





 40%|███▉      | 182724/462567 [00:00<00:00, 452903.16it/s]


In [28]:
df

,title,press,date_upload,mecab_noun_stock,url,stock
0,"이복현#""소상공인#울리는#보이스피싱#엄단""",이데일리,2023-04-05,대웅,https://n.news.naver.com/mnews/article/018/000...,[대웅]
1,이석태·강신국·박완식·조병규…#우리은행장#4파전,디지털타임스,2023-03-26,대교#우리금융지주,https://n.news.naver.com/mnews/article/029/000...,"[대교, 우리금융지주]"
2,"KB금융,#중국#소비회복#가능성#엿본#보고서#발간",스포츠서울,2023-03-26,KB금융,https://n.news.naver.com/mnews/article/468/000...,[KB금융]
3,코람코에#1위#빼앗긴#한토신·한자신…올해도#'먹구름'#전망,더팩트,2023-03-31,한국토지신탁#카카오#한국자산신탁#한국기업평가,https://n.news.naver.com/mnews/article/629/000...,"[한국토지신탁, 카카오, 한국자산신탁, 한국기업평가]"
4,임승태#KDB생명#신임#대표이사#취임,서울경제,2023-03-31,KD#DB#CS,https://n.news.naver.com/mnews/article/011/000...,"[KD, DB, CS]"
...,...,...,...,...,...,...
462560,"국내#전기차#판매#'뒷걸음질'...정부#""보조금#확대""",YTN,2023-09-22,YTN#하이브#카카오,https://n.news.naver.com/mnews/article/052/000...,"[YTN, 하이브, 카카오]"
462561,"김동철#한전#사장#""위기극복#때까지#퇴근#안#할#것""",YTN,2023-09-22,한국전력#YTN#카카오,https://n.news.naver.com/mnews/article/052/000...,"[한국전력, YTN, 카카오]"
462564,국내#기름값#10주째#상승...물가·무역수지#'악재'#우려,YTN,2023-09-16,YTN#카카오,https://n.news.naver.com/mnews/article/052/000...,"[YTN, 카카오]"
462565,예상#웃돈#미국#물가#상승에도...시장#안도한#이유는?,YTN,2023-09-16,현대차#현대차증권#YTN#카카오,https://n.news.naver.com/mnews/article/052/000...,"[현대차, YTN, 카카오]"


In [4]:
# 1000개 이상인거 리스트로 만들기, 길이 확인
import ast
with open('1차편집본.txt', 'r') as f:
    data = f.read()
    list1 = ast.literal_eval(data)
    
df2 = df['stock'].explode().value_counts()
df2[df2>1000].index.tolist()
exception_word = [x for x in df2[df2>1000].index.tolist() if x not in list1]
df['stock'] = df['stock'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
print(len(exception_word), len(df2[df2>1000].index.tolist()))
df.head(3)

24 129


,title,press,date_upload,mecab_noun_stock,url,stock
0,"이복현#""소상공인#울리는#보이스피싱#엄단""",이데일리,2023-04-05,대웅,https://n.news.naver.com/mnews/article/018/000...,대웅
1,이석태·강신국·박완식·조병규…#우리은행장#4파전,디지털타임스,2023-03-26,대교#우리금융지주,https://n.news.naver.com/mnews/article/029/000...,"대교, 우리금융지주"
2,"KB금융,#중국#소비회복#가능성#엿본#보고서#발간",스포츠서울,2023-03-26,KB금융,https://n.news.naver.com/mnews/article/468/000...,KB금융


In [5]:
exception_word

['레이',
 'SK',
 'LG',
 '카카오',
 'SG',
 '테스',
 'KT',
 '디오',
 'CS',
 '태양',
 '전방',
 '남성',
 'KD',
 '나노',
 '신흥',
 '한창',
 '아하',
 '흥국',
 '진도',
 'SBS',
 'JYP Ent.',
 'NAVER',
 '진영',
 'DSR']

In [6]:
# # 제외단어 제외한 개수 확인
import numpy as np
import re

df3 = df.copy()
df3['stock'] = df3['stock'].str.split(',')
df3 = df3.explode('stock').reset_index(drop=True)
for word in exception_word:
    df3['stock'] = df3['stock'].apply(lambda x: x.replace(word, '') if isinstance(x, str) and x.strip() == word else x)
df3 = df3[df3['stock'].str.len() > 0]
df3['stock'].replace('', np.nan, inplace=True)
df3.dropna(subset=['stock'], inplace=True)
df3.tail(2)


,title,press,date_upload,mecab_noun_stock,url,stock
689055,'돈#줄'#마르자#건설사#줄도산‥대형건설사도#흑자도산#위기,MBC,2023-09-22,우진#카카오,https://n.news.naver.com/mnews/article/214/000...,우진
689056,'돈#줄'#마르자#건설사#줄도산‥대형건설사도#흑자도산#위기,MBC,2023-09-22,우진#카카오,https://n.news.naver.com/mnews/article/214/000...,


In [7]:
df4 = df3.groupby('url')['stock'].apply(list).reset_index()
df5 = pd.merge(df3.drop('stock', axis=1), df4, on='url')
# print(len(df5))
df5['date_upload'] = df5['date_upload'].astype(str)  
df5['stock'] = df5['stock'].apply(lambda x: ','.join(x))
df3 = df5.drop_duplicates()
df3.loc[df3['stock'].apply(lambda x : isinstance(x, float)), 'stock'] = np.nan
df3 = df3.dropna(subset=['stock'])
print(len(df3))
df3.head()

178850


,title,press,date_upload,mecab_noun_stock,url,stock
0,"이복현#""소상공인#울리는#보이스피싱#엄단""",이데일리,2023-04-05,대웅,https://n.news.naver.com/mnews/article/018/000...,대웅
1,이석태·강신국·박완식·조병규…#우리은행장#4파전,디지털타임스,2023-03-26,대교#우리금융지주,https://n.news.naver.com/mnews/article/029/000...,"대교, 우리금융지주"
3,"KB금융,#중국#소비회복#가능성#엿본#보고서#발간",스포츠서울,2023-03-26,KB금융,https://n.news.naver.com/mnews/article/468/000...,KB금융
4,코람코에#1위#빼앗긴#한토신·한자신…올해도#'먹구름'#전망,더팩트,2023-03-31,한국토지신탁#카카오#한국자산신탁#한국기업평가,https://n.news.naver.com/mnews/article/629/000...,"한국토지신탁, , 한국자산신탁, 한국기업평가"
8,임승태#KDB생명#신임#대표이사#취임,서울경제,2023-03-31,KD#DB#CS,https://n.news.naver.com/mnews/article/011/000...,"DB,"


In [46]:
# 장바구니 분석을 위한 준비
# 단일데이터 추출(unique)(다시 리스트로 변환)
# df['stock'] = df['stock'].apply(lambda x: [i for i in x.split(', ') if i])
# unique_values = df['stock'].explode().unique()
# unique_list = list(unique_values)
df4 = df.copy()
df4

,title,press,date_upload,mecab_noun_stock,url,stock
0,"이복현#""소상공인#울리는#보이스피싱#엄단""",이데일리,2023-04-05,대웅,https://n.news.naver.com/mnews/article/018/000...,[대웅]
1,이석태·강신국·박완식·조병규…#우리은행장#4파전,디지털타임스,2023-03-26,대교#우리금융지주,https://n.news.naver.com/mnews/article/029/000...,"[대교, 우리금융지주]"
2,"KB금융,#중국#소비회복#가능성#엿본#보고서#발간",스포츠서울,2023-03-26,KB금융,https://n.news.naver.com/mnews/article/468/000...,[KB금융]
3,코람코에#1위#빼앗긴#한토신·한자신…올해도#'먹구름'#전망,더팩트,2023-03-31,한국토지신탁#카카오#한국자산신탁#한국기업평가,https://n.news.naver.com/mnews/article/629/000...,"[한국토지신탁, 카카오, 한국자산신탁, 한국기업평가]"
4,임승태#KDB생명#신임#대표이사#취임,서울경제,2023-03-31,KD#DB#CS,https://n.news.naver.com/mnews/article/011/000...,"[KD, DB, CS]"
...,...,...,...,...,...,...
462560,"국내#전기차#판매#'뒷걸음질'...정부#""보조금#확대""",YTN,2023-09-22,YTN#하이브#카카오,https://n.news.naver.com/mnews/article/052/000...,"[YTN, 하이브, 카카오]"
462561,"김동철#한전#사장#""위기극복#때까지#퇴근#안#할#것""",YTN,2023-09-22,한국전력#YTN#카카오,https://n.news.naver.com/mnews/article/052/000...,"[한국전력, YTN, 카카오]"
462564,국내#기름값#10주째#상승...물가·무역수지#'악재'#우려,YTN,2023-09-16,YTN#카카오,https://n.news.naver.com/mnews/article/052/000...,"[YTN, 카카오]"
462565,예상#웃돈#미국#물가#상승에도...시장#안도한#이유는?,YTN,2023-09-16,현대차#현대차증권#YTN#카카오,https://n.news.naver.com/mnews/article/052/000...,"[현대차, YTN, 카카오]"


In [8]:
df4['stock'] = df3['stock'].str.split(',')
df4 = df4.dropna(subset=['stock'])
df4['stock'] = df4['stock'].apply(lambda x: [item for item in x if item.strip()])
df4['stock'] = df4['stock'].apply(lambda x: np.nan if len(x) == 0 else x)
df4 = df4.dropna(subset=['stock'])
df4.reset_index(drop=True, inplace=True)
df4

,url,stock
0,https://n.news.naver.com/mnews/article/001/001...,[대웅]
1,https://n.news.naver.com/mnews/article/001/001...,"[대교, 우리금융지주]"
2,https://n.news.naver.com/mnews/article/001/001...,[KB금융]
3,https://n.news.naver.com/mnews/article/001/001...,"[한국토지신탁, 한국자산신탁, 한국기업평가]"
4,https://n.news.naver.com/mnews/article/001/001...,[ DB]
...,...,...
40436,https://n.news.naver.com/mnews/article/666/000...,"[대창, 파워로직스, LS전선아시아, 덕성, 모비스, 원익, 국일신동, ..."
40437,https://n.news.naver.com/mnews/article/666/000...,"[DB금융투자, DB]"
40438,https://n.news.naver.com/mnews/article/666/000...,"[SK하이닉스, 엘앤에프, 덕성, 파워로직스, 에코프로, 포스코DX, 삼..."
40439,https://n.news.naver.com/mnews/article/666/000...,"[SK하이닉스, 엘앤에프, 에코프로, 포스코DX, LG에너지솔루션, 삼성바..."


In [9]:
# 장바구니 분석
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori


# Prepare the data
te = TransactionEncoder()
te_ary = te.fit(df4['stock']).transform(df4['stock'], sparse = True)
te_df = pd.DataFrame.sparse.from_spmatrix(te_ary,columns=te.columns_)
# df4 = pd.DataFrame(te_ary, columns=te.columns_)

# Apply Apriori algorithm
# frequent_itemsets = apriori(df5, min_support=0.1, use_colnames=True)
    

In [149]:
te_df

,3S,AJ네트웍스,AK홀딩스,APS,AP시스템,AP위성,BGF,BGF에코머티리얼즈,BYC,CBI,...,휴온스,휴온스글로벌,휴젤,흥구석유,흥국,흥국에프엔비,흥국화재,흥아해운,희림,힘스
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30238,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30239,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30240,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30241,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1]:
import pandas as pd

# 저장할때
# frequent_itemsets['itemsets'] = frequent_itemsets['itemsets'].apply(lambda x: str(x))
# frequent_itemsets.to_csv("frequent_itemsets2.csv", index=False) 
# # 저장된 파일 불러올때
frequent_itemsets = pd.read_csv('frequent_itemsets2.csv', converters={'itemsets': eval})
# frequent_itemsets['itemsets'] = frequent_itemsets['itemsets'].apply(lambda x: ast.literal_eval(x))
cleaned_data = frequent_itemsets.copy()
cleaned_data['itemsets'] = frequent_itemsets['itemsets'].apply(lambda x: frozenset(item.strip() for item in x))
# cleaned_data['itemsets'] = cleaned_data['itemsets'].apply(set)
# frequent_itemsets['itemsets'] = cleaned_data['itemsets']
# cleaned_data = cleaned_data[cleaned_data['itemsets'].apply(lambda x: len(x) >= 2)]
cleaned_data #[cleaned_data['itemsets'] == frozenset({'도이치모터스'}) ]
# type(frequent_itemsets['itemsets'].iloc[0])
# unique_data = cleaned_data.groupby('itemsets').agg({'support': 'mean', 'itemsets': 'first', 'length': 'first'})
# unique_data.drop('itemsets', axis=1, inplace=True)
# unique_data.reset_index(inplace=True)
# unique_data


,support,itemsets,length
0,0.000198,(3S),1
1,0.000297,(AJ네트웍스),1
2,0.000198,(AK홀딩스),1
3,0.000099,(APS),1
4,0.000099,(AP시스템),1
...,...,...,...
1381696,0.000025,"(LG유플러스, 휴온스, 휴온스글로벌)",3
1381697,0.000025,"(아난티, 휴온스, 휴온스글로벌)",3
1381698,0.000025,"(종근당, 휴온스, 휴젤)",3
1381699,0.000049,"(휴온스, 한국비엔씨, 휴젤)",3


In [150]:
# 초기 데이터 저장

frequent_itemsets = apriori(te_df, min_support=0.005, max_len=3, use_colnames=True, verbose=1, low_memory=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].map(lambda x: len(x))
# frequent_itemsets.sort_values('support', ascending=False, inplace=True)


frequent_itemsets

Processing 780 combinations | Sampling itemset size 3


,support,itemsets,length
0,0.006845,( CJ CGV),1
1,0.008035,( CJ ENM),1
2,0.033925,( CS),1
3,0.012763,( DSR),1
4,0.005257,( E1),1
...,...,...,...
356,0.008233,"(HLB, 오스템임플란트, 카카오게임즈)",3
357,0.008200,"(HLB, 펄어비스, 오스템임플란트)",3
358,0.013259,"(HLB, 펄어비스, 카카오게임즈)",3
359,0.007109,"( 포스코DX, HLB, 펄어비스)",3


In [2]:
from mlxtend.frequent_patterns import association_rules
import new
import numpy as np

association_rules_df = new(cleaned_data[cleaned_data['length'] < 3 ], 
                                         metric='confidence'
                                        )
all_confidences = []
collective_strengths = []
cosine_similarities = []
for _,row in association_rules_df.iterrows():
    all_confidence_if = list(row['antecedents'])[0]
    all_confidence_then = list(row['consequents'])[0]
    if row['antecedent support'] <= row['consequent support']:
        all_confidence_if = list(row['consequents'])[0]
        all_confidence_then = list(row['antecedents'])[0]
    all_confidence = {all_confidence_if+' => '+all_confidence_then : \
                      row['support']/max(row['antecedent support'], row['consequent support'])}
    all_confidences.append(all_confidence)
    
    violation = row['antecedent support'] + row['consequent support'] - 2*row['support']
    ex_violation = 1-row['antecedent support']*row['consequent support'] - \
                    (1-row['antecedent support'])*(1-row['consequent support'])
    try:
        collective_strength = (1-violation)/(1-ex_violation)*(ex_violation/violation)
        collective_strengths.append(collective_strength)
    except:
        collective_strength = 0
        collective_strengths.append(collective_strength)
    cosine_similarity = row['support']/np.sqrt(row['antecedent support']*row['consequent support'])
    cosine_similarities.append(cosine_similarity)

association_rules_df['all-confidence'] = all_confidences
association_rules_df['collective strength'] = collective_strengths
association_rules_df['cosine similarity'] = cosine_similarities
association_rules_df['length'] = association_rules_df['antecedents'].map(lambda x: len(x)) + association_rules_df['consequents'].map(lambda x: len(x))



TypeError: 'module' object is not callable

In [310]:
from new_association import new_rules
new_rules_df = new_rules(unique_data[unique_data['length'] < 3 ],only="leverage")
new_rules_df['length'] = new_rules_df['antecedents'].map(lambda x: len(x)) + new_rules_df['consequents'].map(lambda x: len(x))
new_rules_df.sort_values(by='zhangs_metric')

,antecedents,consequents,support,confidence,lift,leverage,zhangs_metric,length
157080,(SK하이닉스),(한화생명),0.000025,0.000589,0.032680,-0.000732,-0.968649,2
157081,(한화생명),(SK하이닉스),0.000025,0.001372,0.032680,-0.000732,-0.967890,2
55048,(에코프로),(한화생명),0.000037,0.000660,0.036601,-0.000976,-0.965386,2
55049,(한화생명),(에코프로),0.000037,0.002058,0.036601,-0.000976,-0.964035,2
76241,(셀트리온헬스케어),(한화생명),0.000025,0.000689,0.038206,-0.000622,-0.963115,2
...,...,...,...,...,...,...,...,...
54135,(에코프로),(탑엔지니어링),0.000074,0.001320,26.681988,0.000071,1.019855,2
67140,(삼성전자),(셀트리온제약),0.021167,0.338608,23.815008,0.020278,1.021889,2
67868,(삼성전자),(일진디스플),0.000074,0.001187,23.995847,0.000071,1.022226,2
67889,(삼성전자),(조선내화),0.000074,0.001187,23.995847,0.000071,1.022226,2


In [15]:
from new_association import new_rules
sorted_rules = new_rules(unique_data[unique_data['length'] < 3 ],only="lift").sort_values(by='confidence', ascending=False)
print(max(sorted_rules['confidence']), max(sorted_rules['zhangs_metric']))
sorted_rules

KeyError: "frozenset({'3S'})You are likely getting this error because the DataFrame is missing  antecedent and/or consequent  information. You can try using the  `support_only=True` option"

,antecedents,consequents,support,confidence,lift,leverage,zhangs_metric
75339,(스마트솔루션즈),(인트로메딕),0.000297,1.846154,4666.269231,0.000297,0.999946
75313,(스마트솔루션즈),(시스웍),0.000297,1.846154,5743.100592,0.000297,0.999987
75539,(스마트솔루션즈),(피에이치씨),0.000297,1.846154,8783.565611,0.000297,1.000047
25906,(만호제강),(대한방직),0.003215,1.830986,507.170558,0.003208,0.999784
113697,(풍국주정),(DB),0.000223,1.800000,49.284902,0.000218,0.979831
...,...,...,...,...,...,...,...
67880,(삼성전자),(일진하이솔루스),0.000025,0.000396,3.999308,0.000019,0.799963
10526,(삼성전자),(비플라이소프트),0.000025,0.000396,6.398892,0.000021,0.899982
67886,(삼성전자),(제이브이엠),0.000025,0.000396,1.683919,0.000010,0.433229
67890,(삼성전자),(조광피혁),0.000025,0.000396,0.941014,-0.000002,-0.062673


In [236]:
# association_rules_df
pivot_df = association_rules_df[association_rules_df['length'] == 2].pivot_table(index='antecedents', columns='consequents', values='confidence', aggfunc='max')
def search_name(name,stock):
    stocks = {1:'support',2:'confidence',3:'lift',4:'leverage',5:'conviction',6:'zhangs_metric',7:'collective strength',8:'cosine similarity'}
    selected_stock = stocks[int(stock)]
    search_n = association_rules_df[
    (association_rules_df['antecedents'] == frozenset({name})) &
    (association_rules_df['length'] == 2) &
    (association_rules_df['zhangs_metric'] <= 1) &  
    (association_rules_df['confidence'] <= 1) &
    (~association_rules_df['consequents'].apply(lambda x: list(x)[0].endswith('우') or list(x)[0].endswith('우B')))
]
    sorted_rules = search_n.sort_values(by=selected_stock, ascending=False)
    top_10_results = sorted_rules[['consequents', selected_stock]].values.tolist()
    consequents_list = []
    for consequent in top_10_results:
        consequent_name = list(consequent[0])[0]
        support_value = round(consequent[1], 5) 
        consequents_list.append([consequent_name, support_value])

    result_df = pd.DataFrame(consequents_list, columns=['종목명','관련도'], index=None)
    result_df.index += 1
    # print(f"{name}에 대한 관련도 입니다.")
    # print("-"*40)
    return result_df

# a = input("변수를 선택해주세요 {1:'support',2:'confidence',3:'lift',4:'leverage',5:'conviction',6:'zhangs_metric',7:'collective strength',8:'cosine similarity'}")
# search_name('에스엠',a)



In [299]:
def find_name(name, metric="confidence"):
    metric = metric
    if metric == 'support':
        find_s = new_rules_df[
            (new_rules_df['antecedents'] == frozenset({name})) &
            (new_rules_df['length'] == 2) &
            (~new_rules_df['consequents'].apply(lambda x: list(x)[0].endswith('우') or list(x)[0].endswith('우B')))
        ].sort_values(by='support', ascending=False)
        support_results = find_s[['consequents', 'support']].values.tolist()
        support_list = []
        for consequent in support_results:
            consequent_name = list(consequent[0])[0]
            support_value = round(consequent[1]/max(new_rules_df['support'], 5))
            support_list.append([consequent_name, support_value])
        result_df = pd.DataFrame(support_list, columns=['종목명','지지도'], index=None)
        print(f"{name}에 대한 지지도 입니다.")
        print("-"*40)
    elif metric == 'zhangs_metric':
        find_z = new_rules_df[
            (new_rules_df['antecedents'] == frozenset({name})) &
            (new_rules_df['length'] == 2) &
            (~new_rules_df['consequents'].apply(lambda x: list(x)[0].endswith('우') or list(x)[0].endswith('우B')))
        ].sort_values(by='zhangs_metric', ascending=False)
        zhangs_metric_results = find_z[['consequents', 'zhangs_metric']].values.tolist()
        zhangs_metric_list = []
        for consequent in zhangs_metric_results:
            consequent_name = list(consequent[0])[0]
            zhangs_metric_value = round(consequent[1]/max(new_rules_df['zhangs_metric']), 5) 
            zhangs_metric_list.append([consequent_name, zhangs_metric_value])
        result_df = pd.DataFrame(zhangs_metric_list, columns=['종목명','zhangs_metric'], index=None)
        print(f"{name}에 대한 zhangs_metric 입니다.")
        print("-"*40)
    else:
        find_c = new_rules_df[
            (new_rules_df['antecedents'] == frozenset({name})) &
            (new_rules_df['length'] == 2) &
            (~new_rules_df['consequents'].apply(lambda x: list(x)[0].endswith('우') or list(x)[0].endswith('우B')))
        ].sort_values(by='confidence', ascending=False)
        confidence_results = find_c[['consequents', 'confidence']].values.tolist()
        confidence_list = []
        for consequent in confidence_results:
            consequent_name = list(consequent[0])[0]
            confidence_value = round(consequent[1]/max(new_rules_df['confidence']), 5) 
            confidence_list.append([consequent_name, confidence_value])
        result_df = pd.DataFrame(confidence_list, columns=['종목명','신뢰도'], index=None)
        print(f"{name}에 대한 신뢰도 입니다.")
        print("-"*40)
    result_df.index += 1

    return result_df.head(10)

In [303]:
find_name('삼성전자',metric='zhangs_metric')

삼성전자에 대한 zhangs_metric 입니다.
----------------------------------------


,종목명,zhangs_metric
1,대모,0.94914
2,오스템,0.94912
3,대덕전자,0.94867
4,셀트리온제약,0.94861
5,LX세미콘,0.94830
6,SFA반도체,0.94830
7,성일하이텍,0.94803
8,서울가스,0.94803
9,이오테크닉스,0.94788
10,천보,0.94784


In [193]:
# search_n = association_rules_df[
#     (association_rules_df['antecedents'] == frozenset({'마니커'})) &
#     (~association_rules_df['consequents'].apply(lambda x: list(x)[0].endswith('우') or list(x)[0].endswith('우B')))
# ]
# for i in range(len(search_n)):
#     if search_n['consequents'].iloc[i] == frozenset({'DB'}):
#         print(1)
#         break
# search_n['consequents'].iloc[0]

In [302]:
max(new_rules_df['confidence'])


157.4999999999989

In [228]:
# for i in range(len(search_name('마니커',4))):
#     if search_name('마니커',4)['마니커'].iloc[i] == '사조씨푸드':
#         print('1')
#         break
#     else:
#         pass
search_name('이수화학',2)

,종목명,관련도
1,삼성전자,0.02540
2,이수스페셜티케미컬,0.01905
3,금양,0.01799
4,LG에너지솔루션,0.01481
5,현대차,0.01164
6,LG화학,0.01164
7,삼성SDI,0.01058
8,자이글,0.00847
9,POSCO홀딩스,0.00741
10,카프로,0.00741


In [197]:
association_rules_df[association_rules_df['length']==2].to_csv('data_all.csv')

In [237]:
pivot_df

consequents,(AJ네트웍스),(조광페인트),(이원컴포텍),(이미지스),(옴니시스템),(엘아이에스),(엔지스테크널러지),(아리온),(슈피겐코리아),(셀바이오휴먼텍),...,(모트렉스),(메드팩토),(메가스터디교육),(메가스터디),(맥스트),(매일홀딩스),(마이크로투나노),(레고켐바이오),(상아프론테크),(극동유화)
antecedents,,,,,,,,,,,,,,,,,,,,,
(JW홀딩스),1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(케어랩스),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(네오팜),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(티쓰리),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(제룡산업),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
(포스코인터내셔널),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(퍼스텍),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(코드네이처),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
max_i = 4
for i, row in association_rules_df.iterrows():
    print("Rule: " + list(row['antecedents'])[0] + " => " + list(row['consequents'])[0])
 
    print("Support: " + str(round(row['support'],2)))
 
    print("Confidence: " + str(round(row['confidence'],2)))
    print("Lift: " + str(round(row['lift'],2)))
    print("=====================================")
    if i==max_i:
        break

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcl
 
  
from matplotlib.colors import LinearSegmentedColormap
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules


support = association_rules_df['support']
confidence = association_rules_df['confidence']
 
h = 347
s = 1
v = 1
colors = [
    mcl.hsv_to_rgb((h/360, 0.2, v)),
    mcl.hsv_to_rgb((h/360, 0.55, v)),
    mcl.hsv_to_rgb((h/360, 1, v))
]
cmap = LinearSegmentedColormap.from_list('my_cmap',colors,gamma=2)
 
measures = ['lift', 'leverage', 'conviction', 
            'all-confidence', 'collective strength', 'cosine similarity']
 
fig = plt.figure(figsize=(15,10))
fig.set_facecolor('white')
for i, measure in enumerate(measures):
    ax = fig.add_subplot(320+i+1)
    if measure != 'all-confidence':
        scatter = ax.scatter(support,confidence,c=association_rules_df[measure],cmap=cmap)
    else:
        scatter = ax.scatter(support,confidence,c=association_rules_df['all-confidence'].map(lambda x: [v for k,v in x.items()][0]),cmap=cmap)
    ax.set_xlabel('support')
    ax.set_ylabel('confidence')
    ax.set_title(measure)
    
    fig.colorbar(scatter,ax=ax)
fig.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()